Default Switch からVMに接続し InternalSwitch に固定IPアドレスを設定する

# 定義

In [ ]:
$vmName           = 'Test-Win11'
$internalSwitch   = 'Internal Switch'
$vmInternalIP     = '192.168.128.100'
$internalSwitchIP = '192.168.128.254'
$prefixLength     = 24
$cred = New-Object System.Management.Automation.PSCredential (
    'sakaztk',
    (ConvertTo-SecureString -AsPlainText -Force -String 'Passw0rd!')
)

# 内部スイッチが無ければ作る

In [ ]:
if (Get-VMSwitch -Name $internalSwitch -ErrorAction SilentlyContinue) {'OK'}
else {
    New-VMSwitch -Name $internalSwitch -SwitchType Internal -Verbose
    New-NetIPAddress -InterfaceAlias "vEthernet ($internalSwitch)" -IPAddress $internalSwitchIP -PrefixLength $prefixLength
}

# VMにスイッチを接続

In [ ]:
if (-not(Get-VMNetworkAdapter -VMName $vmName | Where-Object { $_.SwitchName -eq $internalSwitch })) {
    Add-VMNetworkAdapter -VMName $vmName -SwitchName $internalSwitch -Verbose
}
Get-VMNetworkAdapter -VMName $vmName

# Default Switch のIPアドレスを取得してCimSessionを作成

In [ ]:
$dsIpAddress = (Get-VMNetworkAdapter -VMName $vmName | Where-Object {$_.SwitchName -eq 'Default Switch'}).IPAddresses |
  Where-Object {($_ -match '^\d{1,3}(\.\d{1,3}){3}$') -and ($_ -notlike '169.254*') -and ($_ -ne '0.0.0.0')}
$dsIpAddress

In [ ]:
$cimSession = New-CimSession -ComputerName $dsIpAddress -Credential $cred -Verbose
$cimSession

# 固定IPアドレスを設定
リンクローカルアドレスになっているものにIPアドレスを設定

In [ ]:
Get-NetIPAddress -CimSession $cimSession -AddressFamily IPv4 -IPAddress '169.254.*' |
  New-NetIPAddress -IPAddress $vmInternalIP -PrefixLength $prefixLength -Verbose

# ネットワークプロファイルをプライベートにする

In [ ]:
Get-NetIPAddress -CimSession $cimSession -IPAddress $vmInternalIP |
  Set-NetConnectionProfile -NetworkCategory Private -Verbose -PassThru

# 切断

In [ ]:
Remove-CimSession $cimSession

In [ ]:
$pss = New-PSSession -ComputerName $vmInternalIP -Credential $cred
$pss

In [ ]:
Invoke-Command -Session $pss -Command { Get-NetIPAddress -AddressFamily IPv4}

In [ ]:
$pss | Remove-PSSession